In [ ]:
# To run R in Jupyter Notebook
%load_ext rpy2.ipython

In [ ]:
pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv('.env')

API_KEY = os.getenv('GOOGLE_API_KEY')

%R -i API_KEY


Data Preparation for Air Pollution Index 2018

In [ ]:
%%R
# Data Preparation for Air Pollution Index 2018

API = read.csv("RawDataSet\\API.csv",header=TRUE)       # Read the data
API$Time = strptime(API$Time, format = "%d-%m-%y %H:%M")    # Change the format of Time
API$Time = format(API$Time, "%Y")                                               # Change the time attribute only the year
API$Time = as.integer(API$Time)                                                     # Change the format of Time to integer
API = as.data.frame(lapply(API, as.integer))                                      # I also don't know why I need to do this

API_18 = subset(API, Time == 2018)                    # Subset the data to only 2018
API_18 = round(colMeans(API_18, na.rm = TRUE), digits = 0)
API_18 = data.frame(API_18)

#geocode
library(ggmap)
register_google(key = API_KEY)

hold_coor = data.frame()
hold_coors = data.frame()

for (i in 2:nrow(API_18)) {
  location = paste(rownames(API_18)[i], ", Malaysia")
  coor = geocode(location = location, output = "latlon")
  hold_coor = rbind(hold_coor, cbind(location, coor))
}

API_18 = data.frame(API_18[-1,]) #remove the year row
hold_coors = cbind(hold_coor, API_18)
colnames(hold_coors) = c("location", "lon", "lat", "API")

  # remove east malaysia
hold_coors = subset(hold_coors, hold_coors$lon < 105)
write.csv(hold_coors, file = "PreparedDataSet\\API_18.csv")

Data Prepation for API 2019

In [ ]:
%%R
# Data Preparation for Air Pollution Index 2019

API = read.csv("RawDataSet\\API.csv",header=TRUE)       # Read the data
API$Time = strptime(API$Time, format = "%d-%m-%y %H:%M")    # Change the format of Time
API$Time = format(API$Time, "%Y")                                               # Change the time attribute only the year
API$Time = as.integer(API$Time)                                                     # Change the format of Time to integer
API = as.data.frame(lapply(API, as.integer))                                      # I also don't know why I need to do this

API_19 = subset(API, Time == 2019)                    # Subset the data to only 2019
API_19 = round(colMeans(API_19, na.rm = TRUE), digits = 0)
API_19 = data.frame(API_19)

#geocode
library(ggmap)
register_google(key = API_KEY)

hold_coor = data.frame()
hold_coors = data.frame()

for (i in 2:nrow(API_19)) {
  location = paste(rownames(API_19)[i], ", Malaysia")
  coor = geocode(location = location, output = "latlon")
  hold_coor = rbind(hold_coor, cbind(location, coor))
}

API_19 = data.frame(API_19[-1,]) #remove the year row
hold_coors = cbind(hold_coor, API_19)
colnames(hold_coors) = c("location", "lon", "lat", "API")

  # remove east malaysia & langkawi
hold_coors = subset(hold_coors, hold_coors$lon < 105)
hold_coors = subset(hold_coors, hold_coors$lon > 100)
write.csv(hold_coors, file = "PreparedDataSet\\API_19.csv")

Meteo

In [ ]:
%%R
library(dplyr)
Meteo_18 = read.csv("RawDataSet\\Meteo2018.csv")
Meteo_18 = Meteo_18 %>% group_by(name, longitude, latitude) %>% summarize(temp = mean(temp), humidity=mean(humidity),precip=mean(precip),humidity=mean(humidity), windspeed = mean(windspeed))
write.csv(Meteo_18, file = "PreparedDataSet\\Meteo_18.csv")

In [ ]:
%%R
library(dplyr)
Meteo_19 = read.csv("RawDataSet\\Meteo2019.csv")
Meteo_19 = Meteo_19 %>% group_by(name, longitude, latitude) %>% summarize(temp = mean(temp), humidity=mean(humidity),precip=mean(precip),humidity=mean(humidity), windspeed = mean(windspeed))
write.csv(Meteo_19, file = "PreparedDataSet\\Meteo_19.csv")

Vegetation

In [ ]:
%%R
library(dplyr)
Vegetation1819 = read.csv("RawDataSet\\vegetation.csv")

Vegetation1819 = subset(Vegetation1819, Vegetation1819$threshold == 30)         #select only 30% tree cover
Vegetation1819 = Vegetation1819 %>%
            mutate(treecover2018 = extent_2010_ha - rowSums(.[, 19:26], na.rm = TRUE) + gain_2000.2020_ha) %>%
            mutate(treecover2019 = extent_2010_ha - rowSums(.[, 19:27], na.rm = TRUE) + gain_2000.2020_ha) %>%
            select(subnational1,subnational2,treecover2018, treecover2019)
            

#geocode
library(ggmap)
register_google(key = API_KEY)

hold_coor = data.frame()
hold_coors = data.frame()
for (i in 1:nrow(Vegetation1819)) {
  location = paste(Vegetation1819[i,2], ",", Vegetation1819[i,1],", Malaysia")
  coor = geocode(location = location, output = "latlon")
  hold_coor = rbind(hold_coor, cbind(location, coor))
}
hold_coors = cbind(hold_coor,Vegetation1819[,3:4])
#remove east malaysia & langkawi
hold_coors = subset(hold_coors, hold_coors$lon < 105)
hold_coors = subset(hold_coors, hold_coors$lon > 100)
write.csv(hold_coors,file="PreparedDataSet\\Vegetation1819.csv")

Traffic

In [ ]:
%%R
library(dplyr)
library(tidyr)  # Add this line to load the 'tidyverse' package
Traffic = read.csv("RawDataSet\\Traffic.csv")
# Split the column value into two rows
Traffic <- Traffic %>%
    mutate(Location = strsplit(as.character(LOKASI), "-")) %>%
    unnest(Location) %>%
    mutate(Traffic = rep(Traffic, each = 2))

# Print the modified dataframe
print(Traffic)
